# Benchmark

## 1) Setup

In [ ]:
from typing import List, Union, Dict, Tuple
import pandas as pd
import numpy as np
import os
import pickle
import time
import glob
import subprocess
import json

from ipywidgets import IntProgress

In [ ]:
%load_ext Cython

In [ ]:
ALL_HG19 = False

In [ ]:
DATADIR = './data'
MODELDIR = './models/'
RESULTS_PATH = './results'
REPEATS_TO_SEARCH = [1, 2, 3, 4]
BENCHMARKS = ['hg19/chr1.fa', 'hg38/chr1.fa', 'mm10/chr2.fa', 'hg19/chr20.fa']
if ALL_HG19:
    BENCHMARKS = ['hg19/chr{}.fa'.format(i) for i in range(1,23) ]

In [ ]:
%%cython

cimport cython
cimport numpy as np
@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
def confusion_matrix(np.int8_t[:] true_class, np.int8_t[:] pred_class, long[:,:] output, long length):
    """ calculate confusion matrix"""
    cdef int i
    for i in range(length):
        output[true_class[i],pred_class[i]]+=1

In [ ]:
def preprocess_Y(filename: str, chromosom: str, length: int,
                 repeats_to_search: List[int]) -> np.array:
    """ Reads parse_rm file of repeats to numpy array"""

    Ydata = pd.read_csv(filename, sep='\s+', header=None, index_col=False)
    Ydata.columns = [
        'chromosom', 'begin', 'end', 'repeatnumber', 'repeat', 'class'
    ]
    Ydata = Ydata[Ydata.chromosom == chromosom]
    Ydata.drop('chromosom', axis=1, inplace=True)

    bool_series = None
    for number in repeats_to_search:
        if bool_series is None:
            bool_series = (Ydata['repeatnumber'] == number)
        else:
            bool_series |= (Ydata['repeatnumber'] == number)
    Ydata = Ydata[bool_series]
    Y = np.zeros((len(repeats_to_search) + 1, length), dtype=np.int8)

    def assign_toY(row):
        Y[row['repeatnumber'], row.begin:row.end] = 1

    Ydata.apply(assign_toY, axis=1)
    del Ydata
    return Y.argmax(axis=0).astype(np.int8)

In [ ]:
def file_to_array(filename: str,
                  shape: np.array,
                  dnabrnn: bool = False) -> np.array:
    """Reads dna-brnn or deepgrp file to array"""
    headernames = ["file", "chr", "start", "end", "class"]
    if dnabrnn:
        headernames.pop(0)
    tmp = pd.read_csv(filename, header=None, sep="\t",
                      names=headernames).filter(headernames[-3:], axis=1)
    Y = np.zeros(shape, dtype=np.int8)

    def assign_toY(row):
        Y[row.start:row.end] = row['class']

    tmp.apply(assign_toY, axis=1)
    return Y

In [ ]:
def mcc(C):
    """ MCC implementation based on sklearn"""
    t_sum = C.sum(axis=1, dtype=np.float64)
    p_sum = C.sum(axis=0, dtype=np.float64)
    n_correct = np.trace(C, dtype=np.float64)
    n_samples = p_sum.sum()
    cov_ytyp = n_correct * n_samples - np.dot(t_sum, p_sum)
    cov_ypyp = n_samples**2 - np.dot(p_sum, p_sum)
    cov_ytyt = n_samples**2 - np.dot(t_sum, t_sum)
    return cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)

In [ ]:
def calculate_metrics(predictions_class: np.array, true_class: np.array):
    """Calculated important metrics."""
    nof_labels = len(REPEATS_TO_SEARCH) + 1
    cnf_matrix = np.zeros((nof_labels, nof_labels), dtype=int)
    confusion_matrix(true_class, predictions_class, cnf_matrix,
                     true_class.shape[0])
    true_positive = np.diag(cnf_matrix).astype(float)
    false_positive = (cnf_matrix.sum(axis=0) - true_positive).astype(float)
    false_negative = (cnf_matrix.sum(axis=1) - true_positive).astype(float)
    true_negative = (
        cnf_matrix.sum() -
        (false_positive + false_negative + true_positive)).astype(float)
    metrics = {}
    # Sensitivity, hit rate, recall, or true positive rate
    metrics["TPR"] = true_positive / (true_positive + false_negative)
    # Specificity or true negative rate
    metrics["TNR"] = true_negative / (true_negative + false_positive)
    # Precision or positive predictive value
    metrics["PPV"] = true_positive / (true_positive + false_positive)
    # Negative predictive value
    metrics["NPV"] = true_negative / (true_negative + false_negative)
    # Fall out or false positive rate
    metrics["FPR"] = false_positive / (false_positive + true_negative)
    # False negative rate
    metrics["FNR"] = false_negative / (true_positive + false_negative)
    # False discovery rate
    metrics["FDR"] = false_positive / (true_positive + false_positive)
    # Accuracy
    metrics["ACC"] = (true_positive + true_negative) / \
        (true_positive + false_positive + false_negative + true_negative)
    # F1 -Score
    metrics["F1"] = 2 * metrics["TPR"] * \
        metrics["PPV"] / (metrics["TPR"] + metrics["PPV"])
    metrics["TotalACC"] = (
        true_class == predictions_class).sum() / true_class.shape[0]
    metrics['MCC'] = ((true_positive * true_negative) -
                      (false_positive * false_negative)) / np.sqrt(
                          (true_positive + false_positive) *
                          (true_positive + false_negative) *
                          (true_negative + false_positive) *
                          (true_negative + false_negative))
    metrics['totalMCC'] = mcc(cnf_matrix)
    for key in metrics:
        if isinstance(metrics[key], np.ndarray):
            metrics[key] = metrics[key].tolist()
    metrics['confusionmatrix'] = cnf_matrix.tolist()
    return metrics

In [ ]:
def calculate_all(results, modelfiles, is_dnabrnn=False):
    """ Calculates metrics based on deepgrp or dna-brnn output file"""
    for k in results:
        seqlen = np.load(os.path.join(DATADIR, k + '.gz.npz'))['fwd'].shape[1]
        foldername, chrfile = os.path.split(k)
        filename = os.path.join(DATADIR, foldername) + ".bed"
        chromosom = chrfile.replace('.fa', '')
        Ytrue = preprocess_Y(filename, chromosom, seqlen, REPEATS_TO_SEARCH)
        for model in modelfiles:
            modelname = model.replace(MODELDIR, "")
            predfilename = "{}_{}.fa_{}.tsv".format(foldername, chromosom,
                                                    modelname)
            Ypred = file_to_array(predfilename, Ytrue.shape, is_dnabrnn)
            metrics = calculate_metrics(Ypred, Ytrue)
            results[k][modelname].update(metrics)
            del Ypred
        del Ytrue
    return results

In [ ]:
def benchmark(benchmarks: List[str], modelfiles: List[str], errorfile: str,
              command: List[str]) -> Dict[str, Dict[str, float]]:
    """Benchmark a program"""
    n_runs = len(benchmarks) * len(modelfiles)
    results = dict()
    f = IntProgress(min=0, max=n_runs)
    display(f)

    for k in benchmarks:
        results[k] = {}
        infile = os.path.join(DATADIR, k)
        for model_path in modelfiles:
            modelname = model_path.replace(MODELDIR, '')
            print(k, modelname, end='\t')
            outfile = '{}_{}.tsv'.format(k, modelname).replace('/', '_')
            with open(outfile, 'wb') as file:
                env = os.environ.copy()
                env["TF_XLA_FLAGS"]="--tf_xla_auto_jit=2"
                #env["CUDA_VISIBLE_DEVICES"] = ""
                start_time = time.time()
                process = subprocess.Popen(command + [model_path, infile],
                                           stdout=file,
                                           stderr=subprocess.PIPE,
                                           env=env)
                _, errdata = process.communicate()
                end_time = time.time()
            runtime = end_time - start_time
            results[k][modelname] = {'runtime': runtime}
            with open(errorfile, 'ab') as file:
                file.write(errdata)
            f.value += 1
            print(runtime)
    return results

## 2) Run DeepGRP benchmark

In [ ]:
deepgrpmodels = glob.glob(os.path.join(MODELDIR, 'model_*.h5'))
deepgrp_command = ['python3', '-m', 'deepgrp', '-t 10', "--xla", "-b 4096"]
deepgrp_results = benchmark(BENCHMARKS, deepgrpmodels, 'deepgrp.log',
                            deepgrp_command)

### 2.1) Calculate metrics

In [ ]:
deepgrp_results = calculate_all(deepgrp_results, deepgrpmodels)

#### Save results

In [ ]:
filename = 'deepgrp_gpu_results.json'
if ALL_HG19:
    filename = 'deepgrp_results_hg19complete.json'

In [ ]:
with open(os.path.join(RESULTS_PATH, filename), 'w') as file:
    json.dump(deepgrp_results, file)

## 3) Run dna-brnn benchmark 

In [ ]:
dnabrnnmodels = glob.glob(os.path.join(MODELDIR, 'dnabrnn_model*.knm'))
dnabrnn_command = ['dna-nn/dna-brnn', '-t 10', '-O292', '-Ai']
dnabrnnresults = benchmark(BENCHMARKS, dnabrnnmodels, 'dnabrnn.log',
                           dnabrnn_command)

## 3.1) Calculate metrics

In [ ]:
dnabrnnresults = calculate_all(dnabrnnresults, dnabrnnmodels, is_dnabrnn=True)

#### Save results

In [ ]:
filename = 'dnabrnn_results.json'
if ALL_HG19:
    filename = 'dnabrnn_results_hg19complete.json'

In [ ]:
with open(os.path.join(RESULTS_PATH, filename), 'w') as file:
    json.dump(dnabrnnresults, file)